# Tutorial 2: Influence of magnetic field on FM and AFM skyrmions size 

In this tutorial, we will investigate the effect of an external magnetic field pointing in the z direction on the radius of both FM skyrmion and AFM skyrmion on a square lattice.

The Heisenberg Hamiltonian includes an additional term, which is the Zeeman term that is colored in red:

$$
H = - \sum_{\langle i, j \rangle} J_{ij} \mathbf{S}_i \cdot \mathbf{S}_j  
-  \sum_{\langle i,j \rangle} D_{ij} \cdot  (\mathbf{S}_i \times \mathbf{S}_j)
- \sum_i K_i (S_i^z)^2 
- \textcolor{red}{\sum_i \mathbf{B} \cdot \mathbf{S}_i}
$$

The last term represents the Zeeman interaction.



## FM skyrmion

The starting point is the FM skyrmion shown below, where the spins are saved in **FM_skyrmion.ovf** spin file.
The magnetic interactions among the atoms are as shown below, where the DMI vector components stabilize the Bloch skyrmion as shown in Tutorial one. 


<p align="center">
    <img src="./assets/bloch.png" alt="Bloch skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:20%;">
</p>

The magnetic interactions among the atoms are as shown below: 

| Interaction | Value (meV) |
|------------|-------------|
| J        | 4           |
| D        | 0.4         |
| K         | 0.12       |


The interactions are read in the input file from an external file **Square_FM.txt**

In [ ]:
!sed -n '48,49p' input_FM_skyrmion.cfg

By running the Python script **"Magnetic_field_FM_skyrmion.py"**, the script performs the following steps:

1. Reads the input file **"input_FM_skyrmion.cfg"**.
2. Loads the initial magnetic state from **"FM_skyrmion.ovf"**.
3. Applies an out-of-plane (OOP) magnetic field, varying from **0.0 to 2.4 meV** in steps of **0.2 meV**.
4. Starts the Atomistic Spin Dynamics (ASD) simulation to minimize the system's energy.
5. Writes the converged spin configurations as **"spins_B{B:.2f}.ovf"**.

In [ ]:
from Magnetic_field_FM_skyrmion import *

# Simulation setup
cfgfile = "input_FM_skyrmion.cfg"
quiet = True
B_values = np.arange(0.00, 2.4, 0.2)
radii = []

for B in B_values:
    with state.State(cfgfile, quiet) as p_state:
        io.image_read(p_state, "FM_skyrmion.ovf")
        hamiltonian.set_field(p_state, B, [0, 0, 1], idx_image=-1, idx_chain=-1)
        simulation.start(p_state, simulation.METHOD_LLG, simulation.SOLVER_LBFGS_OSO)
        system.update_data(p_state)
        spin_file = f"spins_B{B:.2f}.ovf"
        io.image_write(p_state, spin_file)
        positions = geometry.get_positions(p_state)
        spins = np.loadtxt(spin_file)

        profile, radius, popt = get_profile(positions, spins, f"B{B:.2f}")
        radii.append(radius)

# Filter out zero radius values before plotting
valid_B_values = []
valid_radii = []

for B, radius in zip(B_values, radii):
    if radius > 0:  # Exclude zero radius values
        valid_B_values.append(B)
        valid_radii.append(radius)

# Plot radius vs B value only for nonzero radii
plt.figure(figsize=(10, 6))
plt.plot(valid_B_values, valid_radii, marker='o')
plt.title('Radius vs B Value')
plt.xlabel('B Value')
plt.ylabel('Radius (Angs)')
plt.grid()
plt.savefig('radius_vs_B.png')
plt.show()

6. Plots the resulting magnetic state at each magnetic field value, indicating the skyrmion radius if it still exists as shown in the figure below.

<p align="center">
    <img src="./assets/Bloch-example.png" alt="Bloch skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>


   
7. Generates a final plot of **magnetic field strength vs. skyrmion radius**.

<p align="center">
    <img src="./assets/raduis.png" alt="Bloch skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>


# AFM skyrmion:
In this part, we do the same as has been done in the first part, but here we apply the magnetic field on an AFM skyrmion.
The interactions for this example are as shown below, which are needed to satbilize a Néel  AFM skyrmion on a square lattice.
The interactions are saved in **Square_AFM.txt** file  


 
| Interaction | Value (meV) |
|------------|-------------|
| J1        | -6           |
| J2        | 2          |
| D1        | 0.02         |
| D2        | 0.08           |
| K         | 0.006        |


Those interactions stabilize a Néel skyrmion on a square lattice as shown below: 


<p align="center">
    <img src="./assets/AFM_skyrmion.png" alt="AFM skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>


By executing the Python script **Magnetic_field_AFM_skyrmion.py**, you can observe the impact of the out-of-plane (OOP) magnetic field on the radius of AFM skyrmions by examining the plots corresponding to different magnetic field strengths.

<p align="center">
    <img src="./assets/afm_sky-B.png" alt="AFM skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>